QUESTION 1

Obtain the 200 top-ranking universities in www.topuniversities.com (ranking 2018). In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Some information is not available in the main list and you have to find them in the details page. Store the resulting dataset in a pandas DataFrame and answer the following questions:

Which are the best universities in term of:
    (a) ratio between faculty members and students?
    (b) ratio of international students?

Answer the previous question aggregating the data by (c) country and (d) region

In [32]:
#importing the libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

In [94]:
#Defining a function that will get the attribute of the universiy once we have the 
def Get_data( HTML_tag ) :
    #Using BeautifulSoup to extract the data from the given HTML
    Page_Soup = BeautifulSoup(requests.get(HTML_tag).text, 'html.parser')
    name = Page_Soup.find('h1').text
   
    country = Page_Soup.find('span' , class_='country').text
    #region = Page_Soup.find('span' , class_='locality').text
    


    #We get the number, delete the comma and then the rest of the text (there was spaces and text we had no use of)
    if Page_Soup.find('div', class_='total faculty') is not None:
        faculty_total = re.findall("\d+", Page_Soup.find('div', class_='total faculty').text.replace("," , ""))
        number_faculty_total = int(faculty_total[0])
    else:
        number_faculty_total = 'Unknown'
    
    if Page_Soup.find('div', class_='inter faculty') is not None:
        faculty_international = re.findall("\d+", Page_Soup.find('div', class_='inter faculty').text.replace("," , ""))
        number_faculty_international = int(faculty_international[0])
    else:
        number_faculty_international = 'Unknown'
    
    if Page_Soup.find('div', class_='total student') is not None:
        students_total = re.findall("\d+", Page_Soup.find('div', class_='total student').text.replace("," , ""))
        number_students_total = int(students_total[0])
    else:
        number_students_total = 'Unknown'
        
    if Page_Soup.find('div', class_ = 'total inter') is not None:
        students_international = re.findall("\d+",Page_Soup.find('div', class_ = 'total inter').text.replace("," , ""))
        number_students_international = int(students_international[0])
    else:
        number_students_international = 'Unknown'
    
    #Appending the results to a list
    Data = pd.DataFrame({'name': [name], 'country': [country], 'numb_fac_tot': [number_faculty_total], 
                         'num_fac_int': [number_faculty_international], 'num_stud_tot': [number_students_total], 
                         'num_stud_int': [number_students_international]})
    return Data

In [95]:
Test = Get_data('https://www.topuniversities.com/universities/university-michigan#wurs')
print(Test)

         country                    name  num_fac_int  num_stud_int  \
0  United States  University of Michigan         1920          7527   

   num_stud_tot  numb_fac_tot  
0         43147          6809  


All the data from the universities are not on the main page. However all the links are on the page. We will create a function to exctract all the URLs of each universitiy. The use of a function is handy because later in this exercise we will have to extract the same kind of data from another ranking

In [5]:
#Because we want to extract the data from two distinct website, we will create a boolean parameter as an input of our function
#if the entry is QS, it will extract the data from the corresponding website and the same statement is also true for the other
#one.
def Extract_Links(Original_HTML):

    #Defining a DataFrame that will contain the results:
    columns = ['name', 'country', 'number_faculty_total','number_faculty_international',
                            'number_students_total', 'number_students_international']
    Data = pd.DataFrame(columns=columns)
    
    soup = BeautifulSoup(requests.get(Original_HTML).text, 'html.parser')
    all_links = list(soup.find_all('a'))
    

    #print(np.array(all_links).shape)
    #print(type(all_links))
    #print(all_links)
    #We need to remove all the links which we are not interested in:
    
    print(len(all_links))
    compteur = 1
    
    for link in all_links:
        if link.get('href') is None : 
            all_links.remove(link)
        else:
            #we remove every link that is not leading to a university.
            if (not link.get('href').startswith('https://www.topuniversities.com/universities/')
                and link.get('href').startswith('https://www.topuniversities.com')):
                all_links.remove(link)

            #we remove every link that is not leading to a university.
            if (not link.get('href').startswith('https://www.timeshighereducation.com/world-university-rankings/')
                and link.get('href').startswith('https://www.timeshighereducation.com')):
                all_links.remove(link)
        compteur +=1    
    print('outside the TIMPE')    
    #Appending the result in a DataFrame.
    for FinalLink in all_links:
        Data = Data.append(Get_data(FinalLink))
    #The ranking is not defined, however the universities were stored in order of appearance on the website, meaning that the 
    #DataFrame is sorted by ranking. We define a new column and use it as the index.

    # DONT FORGET TO ADD A COLUMN WITH THE RANK
    return Data

### Extracting data from topuniversities.com

In [96]:
# extract request url from jsonfile and uses them to extract other data via Get_Data()
# takes aloooooot of time

# Used postman to request htmlfile. then searched for <tbody> it was empty so we searched for json/txt to find where
# the documents that JQuery/Ajax or whatever loads in after a while
r = requests.get('https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508507889334')
req = r.json()
data = req['data']
QSframe = pd.DataFrame.from_dict(data)
QSframe = frame.head(201)
QSframe.url = "https://www.topuniversities.com" + frame.url

QSdf = pd.DataFrame()

for i, url in enumerate(frame.url):
    print(i)
    #print(url)
    df = df.append(Get_data(url))


0
https://www.topuniversities.com/universities/massachusetts-institute-technology-mit
1
https://www.topuniversities.com/universities/stanford-university
2
https://www.topuniversities.com/universities/harvard-university
3
https://www.topuniversities.com/universities/california-institute-technology-caltech
4
https://www.topuniversities.com/universities/university-cambridge
5
https://www.topuniversities.com/universities/university-oxford
6
https://www.topuniversities.com/universities/ucl-university-college-london
7
https://www.topuniversities.com/universities/imperial-college-london
8
https://www.topuniversities.com/universities/university-chicago
9
https://www.topuniversities.com/universities/eth-zurich-swiss-federal-institute-technology
10
https://www.topuniversities.com/universities/nanyang-technological-university-singapore-ntu
11
https://www.topuniversities.com/universities/ecole-polytechnique-f%C3%A9d%C3%A9rale-de-lausanne-epfl
12
https://www.topuniversities.com/universities/princet

111
https://www.topuniversities.com/universities/uppsala-university
112
https://www.topuniversities.com/universities/university-groningen
113
https://www.topuniversities.com/universities/universiti-malaya-um
114
https://www.topuniversities.com/universities/nanjing-university
115
https://www.topuniversities.com/universities/nagoya-university
116
https://www.topuniversities.com/universities/technical-university-denmark
117
https://www.topuniversities.com/universities/university-california-davis
118
https://www.topuniversities.com/universities/aarhus-university
119
https://www.topuniversities.com/universities/humboldt-universit%C3%A4t-zu-berlin
120
https://www.topuniversities.com/universities/universidade-de-s%C3%A3o-paulo
121
https://www.topuniversities.com/universities/universidad-nacional-aut%C3%B3noma-de-m%C3%A9xico-unam
122
https://www.topuniversities.com/universities/hokkaido-university
123
https://www.topuniversities.com/universities/wageningen-university
124
https://www.topunivers

In [105]:
#manipulating rank(=21 -> 21) and adding ranking and region from "frame"
QS = QSdf
QS.index = range(len(uni))

QS['region'] = frame['region']
QS['rank'] = frame['rank_display']
print(QS.head(8))


          country                                          name num_fac_int  \
0   United States  Massachusetts Institute of Technology (MIT)         1679   
1   United States                           Stanford University        2042   
2   United States                            Harvard University        1311   
3   United States  California Institute of Technology (Caltech)         350   
4  United Kingdom                       University of Cambridge        2278   
5  United Kingdom                          University of Oxford        2964   
6  United Kingdom               UCL (University College London)        2554   
7  United Kingdom                       Imperial College London        2071   

  num_stud_int num_stud_tot numb_fac_tot         region rank  
0         3717        11067         2982  North America    1  
1         3611        15878         4285  North America    2  
2         5266        22429         4350  North America    3  
3          647         2255         

### Extracting data from timeshighereducation.com

In [113]:
# Used postman to request htmlfile. then searched for <tbody> it was empty so we searched for json/txt to find where
# the documents that JQuery/Ajax or whatever loads in after a while

r = requests.get('https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json')
req = r.json()
data = req['data']
THframe = pd.DataFrame.from_dict(data)
THframe
print(frame)

# seems like alot of the internationalstudents and stuff is there...but is everything we need there?
# if not we must get it with Get_data() or something similar


                                                aliases            location  \
0                                  University of Oxford      United Kingdom   
1                               University of Cambridge      United Kingdom   
2            California Institute of Technology caltech       United States   
3                                   Stanford University       United States   
4                 Massachusetts Institute of Technology       United States   
5                                    Harvard University       United States   
6                                  Princeton University       United States   
7                               Imperial College London      United Kingdom   
8                                 University of Chicago       United States   
9     ETH Zurich – Swiss Federal Institute of Techno...         Switzerland   
10                           University of Pennsylvania       United States   
11                                      Yale Univers

The ranking is not defined, however the universities were stored in order of appearance on the website, meaning that the 
DataFrame is sorted by ranking. We define a new column and use it as the index.

In [6]:
#Redundant
'''
def alternative(link): 
    
    r = requests.get(link)
    
    top = r.json()['data'][:199]
    for t in top:
        print(t['title'],t['rank_display'],t['url'])
'''

In [101]:
#QS_Rank = Extract_Links('https://www.topuniversities.com/university-rankings/world-university-rankings/2018')
#QS_Rank
#QS_Rank.head(20)
#alternative('https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508507889334')

The functions are defined, it is therefore possible to apply the new operations that were required in the question.

In [102]:
html = html.replace('class="row bigbox container mi-df-local locked-single"', 'class="row bigbox container mi-df-local single-local"')
soup = BeautifulSoup(html, "html.parser")
print(html)

NameError: name 'html' is not defined

In [9]:
QS_Rank['faculty/students'] = QS_Rank['number_faculty_total']/QS_Rank['number_students_total']
QS_Rank.set_index('faculty/students')

print(QS_Rank.head(20))
print(TimesHigher_Rank.head(20))

NameError: name 'QS_Rank' is not defined

    (b) ratio of international students

    (a) ratio between faculty members and students

In [104]:
QS_Rank['international/total'] = QS_Rank['number_students_international']/QS_Rank['number_students_total']
QS_Rank.set_index('international/total')
print(QS_Rank.head(20))

NameError: name 'QS_Rank' is not defined

In order to aggregate the results by country, we will use the groupBy function.

    (2) Obtain the 200 top-ranking universities in www.timeshighereducation.com (ranking 2018). Repeat the analysis of the previous point and discuss briefly what you observed.

In [103]:
#Applying the previously defined functions on the other ranking:
TimesHigher_Rank = Extract_Links('https://www.timeshighereducation.com/world-university-rankings/2018/world-ranking#!/page/0/length/25/sort_by/rank/sort_order/asc/cols/stats')
print(TimesHigher_Rank.head(20))

104
outside the TIMPE


TypeError: unsupported operand type(s) for -: 'str' and 'int'

Answering question (a), the faculty members over students ratio:

In [12]:
TimesHigher_Rank['faculty/students'] = TimesHigher_Rank['number_faculty_total']/TimesHigher_Rank['number_students_total']
TimesHigher_Rank.set_index('faculty/students')
print(TimesHigher_Rank.head(20))

NameError: name 'TimesHigher_Rank' is not defined

Answering question (b), the international students over total ratio:

In [13]:
TimesHigher_Rank['international/total'] = TimesHigher_Rank['number_students_international']/TimesHigher_Rank['number_students_total']
TimesHigher_Rank.set_index('international/total')
print(TimesHigher_Rank.head(20))

NameError: name 'TimesHigher_Rank' is not defined